In [1]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [2]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
convMolFeat = deepchem.deepchem.feat.ConvMolFeaturizer()
xgb_reg_ecfp = xgboost.XGBRegressor()
xgb_reg_estate = xgboost.XGBRegressor()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('Caco2_Wang') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    #validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    #trainFeat = np.stack(np.array(trainMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    #validFeat = np.stack(np.array(validMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    train_valFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #featurize training, valid, and test data for xgboost
    #ecfp_f_train = ecfpFeat.featurize(train.iloc[:,1].to_list())
    #ecfp_f_valid = ecfpFeat.featurize(valid.iloc[:,1].to_list())
    ecfp_f_test = ecfpFeat.featurize(test.iloc[:,1].to_list())
    train_val_f = ecfpFeat.featurize(train_val.iloc[:,1].to_list())

    #featurize training, valid, and test data for the GCN
    cv_f_train = convMolFeat.featurize(train.iloc[:,1].to_list())
    cv_f_valid = convMolFeat.featurize(valid.iloc[:,1].to_list())
    cv_f_test = convMolFeat.featurize(test.iloc[:,1].to_list())

    #convert training and validation data into a deepchem dataset for the gcn
    gcn_train_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    gcn_valid_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    #fit data on GCN
    reg = deepchem.deepchem.models.GraphConvModel(
        n_tasks=1, 
        dropout=.0005,
        dense_layer_size=1063,
        graph_conv_layers=[128, 128, 128],
        mode="regression",)
    callback = deepchem.deepchem.models.ValidationCallback(gcn_valid_data, 1000, metric)
    reg.fit(gcn_train_data, nb_epoch=100, callbacks=callback)

    #predict values on gcn and reshape array (reshape value is 20% of dataset size)
    gcn_pred = reg.predict_on_batch(X=np.array(cv_f_test)).reshape(182,)

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg_estate.fit(X=train_valFeat, y=train_val.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_reg_ecfp.fit(X=train_val_f, y=train_val.iloc[:,2], eval_metric="mae")
    pred_estate = xgb_reg_estate.predict(X=testFeat)
    pred_ecfp = xgb_reg_ecfp.predict(X=ecfp_f_test)
    
    #pred_ak = akReg.predict(rdkit_f_test)

    # store test predictions in y_pred_test
    y_pred_test = np.mean([ pred_estate, pred_ecfp, gcn_pred ], axis=0)

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 728/728 [00:00<00:00, 1195.39it/s]
generating training, validation splits...


predictions for run #
1
[-5.771012  -5.046923  -4.6453915 -4.920735  -5.0959873]


100%|██████████| 728/728 [00:00<00:00, 1309.13it/s]
generating training, validation splits...


predictions for run #
2
[-5.600657  -5.3811393 -4.7251935 -5.0375338 -4.929484 ]


100%|██████████| 728/728 [00:00<00:00, 1560.73it/s]


...

predictions for run #
3
[-5.7573266 -5.13537   -4.807997  -4.929074  -4.974685 ]


100%|██████████| 728/728 [00:00<00:00, 1351.20it/s]
generating training, validation splits...


predictions for run #
4
[-5.829679  -4.932092  -4.635282  -4.8561287 -4.9756303]


100%|██████████| 728/728 [00:00<00:00, 1462.16it/s]


predictions for run #
5
[-5.628306  -5.0700135 -4.8060203 -4.9077096 -5.0281405]
Prediction List:
[{'caco2_wang': array([-5.771012 , -5.046923 , -4.6453915, -4.920735 , -5.0959873,
       -5.237889 , -4.5879326, -4.552511 , -4.5664997, -4.575129 ,
       -5.6537967, -5.5742555, -5.5554657, -5.0212235, -4.983809 ,
       -4.916327 , -6.1348686, -5.73414  , -5.5837593, -5.8737144,
       -5.448843 , -5.448843 , -6.1043663, -4.60896  , -5.4166527,
       -5.1214275, -5.971646 , -5.4688015, -5.4680004, -6.7522545,
       -6.7266536, -6.848917 , -4.6554017, -6.7282276, -6.3253207,
       -5.5226784, -6.3253207, -5.246919 , -5.2127233, -5.4008446,
       -5.2327175, -5.1732697, -5.2602215, -5.341051 , -4.7905774,
       -4.7905774, -4.7905774, -4.7905774, -5.03299  , -4.887938 ,
       -4.7500954, -5.03299  , -4.8879385, -4.7500954, -4.7500954,
       -4.8879385, -5.10747  , -5.0639095, -5.073127 , -5.10747  ,
       -5.073127 , -5.0639095, -5.0639095, -5.0731273, -5.0639095,
       -5.07312

In [3]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'caco2_wang': {'mae': 0.397}}
{'caco2_wang': {'mae': 0.437}}
{'caco2_wang': {'mae': 0.389}}
{'caco2_wang': {'mae': 0.362}}
{'caco2_wang': {'mae': 0.344}}


{'caco2_wang': [0.386, 0.032]}